In [3]:
import pandas as pd

In [2]:
# get df_bench and df_all from correct transcription and model transcription
import xml.etree.ElementTree as ET 

def get_df_eaf(eaf_file):
    save_dir = eaf_file.replace('.eaf', '.csv')
    tree = ET.parse(eaf_file)
    root = tree.getroot() 
    
    df_1 = pd.DataFrame()
    TIME_SLOT_ID = []
    TIME_VALUE = []
    for child in root[1]:
        #print(child.tag, child.attrib)
        TIME_SLOT_ID.append(child.attrib['TIME_SLOT_ID'])
        TIME_VALUE.append(child.attrib['TIME_VALUE'])
    df_1['TIME_SLOT_ID'] = TIME_SLOT_ID
    df_1['TIME_VALUE'] = TIME_VALUE
    
    df_2 = pd.DataFrame()
    ANNOTATION_ID = []
    TIME_SLOT_REF1 = []
    TIME_SLOT_REF2 = []
    ANNOTATION_VALUE = []
    for child in root[2]:
        #print(child[0].tag, child[0].attrib)
        #print(child[0][0].text)
        ANNOTATION_ID.append(child[0].attrib['ANNOTATION_ID'])
        TIME_SLOT_REF1.append(child[0].attrib['TIME_SLOT_REF1'])
        TIME_SLOT_REF2.append(child[0].attrib['TIME_SLOT_REF2'])
        ANNOTATION_VALUE.append(child[0][0].text)
    df_2['ANNOTATION_ID'] = ANNOTATION_ID
    df_2['TIME_SLOT_REF1'] = TIME_SLOT_REF1
    df_2['TIME_SLOT_REF2'] = TIME_SLOT_REF2
    df_2['ANNOTATION_VALUE'] = ANNOTATION_VALUE

    df_3 = pd.DataFrame()
    ANNOTATION_ID = []
    ANNOTATION_REF = []
    ANNOTATION_VALUE = []
    for child in root[3]:
        #print(child[0].tag, child[0].attrib)
        #print(child[0][0].text)
        ANNOTATION_ID.append(child[0].attrib['ANNOTATION_ID'])
        ANNOTATION_REF.append(child[0].attrib['ANNOTATION_REF'])
        ANNOTATION_VALUE.append(child[0][0].text)
    df_3['ANNOTATION_ID'] = ANNOTATION_ID
    df_3['ANNOTATION_REF'] = ANNOTATION_REF
    df_3['ANNOTATION_VALUE'] = ANNOTATION_VALUE
    
    df_all = pd.DataFrame()
    start = []
    end = []
    transcription = []
    confidence = []

    for i in range(len(df_2)):
        trans_text = df_2['ANNOTATION_VALUE'][i]
    
        start_id = df_2['TIME_SLOT_REF1'][i]
        end_id = df_2['TIME_SLOT_REF2'][i]
        anno_id = df_2['ANNOTATION_ID'][i]
    
    
        start_time = df_1.loc[df_1['TIME_SLOT_ID'] == start_id]['TIME_VALUE'].tolist()[0]
        end_time = df_1.loc[df_1['TIME_SLOT_ID'] == end_id]['TIME_VALUE'].tolist()[0]
        confidence_value = df_3.loc[df_3['ANNOTATION_REF'] == anno_id]['ANNOTATION_VALUE'].tolist()[0]
    
        transcription.append(trans_text)
        start.append(float(start_time)/1000.0)
        end.append(float(end_time)/1000.0)
        confidence.append(confidence_value)
    
    df_all['start'] = start
    df_all['end'] = end
    df_all['transcription'] = transcription
    df_all['confidence'] = confidence
    df_all.to_csv(save_dir,index=False)
    return df_all

In [37]:
def get_bench_df(eaf_file):
    textgrid_dir = "/users/irisz/downloads/elic_test/"
    file_name = eaf_file.split('/')[-1].replace('.eaf', '.TextGrid')
    textgrid_dir = textgrid_dir + file_name
    
    df_bench = get_audio_df(textgrid_dir)
    #df_bench['text_clean'] = df_bench['text'].apply(lambda x: get_clean_word_list(x))
    df_bench_1 = df_bench[df_bench['tier'].isin([1,2])]
    
    #df_bench_1 = df_bench_1[df_bench_1.text_clean.astype(bool)]
    df_bench_1 = df_bench_1[['text', 'xmin', 'xmax', 'tier']]
    df_bench_1 = df_bench_1.reset_index()
    return df_bench_1
    
def get_merge_loc(df_bench_1, df_all):
    cand_row_min = 0
    cand_max_max = len(df_bench_1)

    merge_loc = {}

    for i in range(len(df_all)):
        # get row time range
        start_curr = df_all['start'][i]
        end_curr = df_all['end'][i]
        for j in range(cand_row_min, cand_max_max):
            # if word window land in the sentence one, put in the merge info
            end_merge_curr = df_bench_1['xmax'][j]
            if start_curr < end_merge_curr:
                merge_loc[i] = j
                cand_row_min = j
                break
    return merge_loc


In [5]:
# get force-aligned wer

# correct df
import numpy as np

import re

def clean_text(text):
    if text == 'x':
        return ''
    # remove () pattern, will leave a ) to be removed
    text = re.sub(r'\(([^)]+)\)', '', text)
    special_char_list = [',', '.', '?', '#', '@', '-', ')']
    for char in special_char_list:
        text = text.replace(char, '')
    # change all text to lower cases
    text = text.lower()
    return text

def get_clean_word_list(text):
    new_text = clean_text(text)
    word_list = new_text.split(' ')
    word_list_clean = list(filter(None, word_list))
    return word_list_clean
    
def get_audio_df(textgrid_dir):
    try:
        with open(textgrid_dir, 'r', encoding='utf-16') as file:
            lines = [line.rstrip() for line in file]
    except:
        with open(textgrid_dir, 'r') as file:
            lines = [line.rstrip() for line in file]
       
    text_list = []
    xmin_list = []
    xmax_list = []
    interval_list = []
    for line in lines[14:]:  #informations needed begin on the 9th lines
        if 'text =' in line:
            line = line.split('"')[1]
            if line != "":
                text_list.append(line)
                #print (len(text_list), len(xmin_list), len(xmax_list), len(interval_list), line, interval_list[-2:])
            else:
                xmin_list = xmin_list[:-1]
                xmax_list = xmax_list[:-1]
                interval_list = interval_list[:-1]
        if 'xmin' in line:
            time_min = line.split('=')[1].strip(' ')
            xmin_list.append(time_min)
        if 'xmax' in line:
            time_max = line.split('=')[1].strip(' ')
            xmax_list.append(time_max)
        if 'intervals [' in line:
            interval = line.split('[')[1].strip(']:')
            interval_list.append(interval)
        if 'intervals: size' in line:
            xmin_list = xmin_list[:-1]
            xmax_list = xmax_list[:-1]
    xmin_list = [float(item) for item in xmin_list]
    xmax_list = [float(item) for item in xmax_list]
    interval_list = [int(item) for item in interval_list]

    df = pd.DataFrame()
    df['text'] = text_list
    df['xmin'] = xmin_list
    df['xmax'] = xmax_list
    df['interval'] = interval_list
    df['tier'] = get_interval(df['interval'].tolist())
    df['start'] = df['xmin'].apply(lambda x: int(np.floor(x*16000)))
    df['end'] = df['xmax'].apply(lambda x: int(np.ceil(x*16000)))
    return df
def get_interval(time_list):
    interval_all = []
    interval_curr = 1
    for i in range(len(time_list)):
        if i == 0:
            interval_all.append(1)
        else:
            if time_list[i] > time_list[i - 1]:
                interval_all.append(interval_curr)
                continue
            else:
                interval_curr = interval_curr + 1
                interval_all.append(interval_curr)
    return interval_all

In [6]:
from Bio import pairwise2
from thefuzz import fuzz
import numpy as np

def get_aligned_list(l1, l2):
    # get aligned list, along with wer type and fuzz match score
    alignments = pairwise2.align.globalxx(l1, 
                                      l2,
                                      gap_char=['-']
                                     )
    l1_align = alignments[0].seqA
    l2_align = alignments[0].seqB
    score = []
    for i in range(len(l1_align)):
        score.append(word_match_score(l1_align[i], l2_align[i]))
    return l1_align, l2_align, score


def word_match_score(t1, t2):
    if t1 == '-' or t2 == '-':
        return 0
    else:
        score = fuzz.ratio(t1, t2)
        return score

def get_matched(t1, t2):
    #print (t1, t2)
    if t1 == t2:
            return 1
    else:
        return 0
    
def get_unmatched_cand(match_list, idx):
    cand = []
    idx_org = idx
    if match_list[idx] == 1:
        return [idx]
    else:
        while match_list[idx] != 1 and idx > 0:
            cand.append(idx)
            idx = idx - 1 # before
        idx = idx_org
        while match_list[idx] != 1 and idx < len(match_list) -1:
            cand.append(idx)
            idx = idx + 1 # after
        return list(set(cand))
    
def get_can_score(idx,match_cand, df_align):
    if len(match_cand) == 1:
        return [100]
    score_all = []
    for cand in match_cand:
        t1 = df_align['manual'][idx]
        t2 = df_align['model'][cand]
        score = word_match_score(t1, t2)
        score_all.append(score)
    return score_all

def get_fuzzy_cand(match_cand, match_cand_score, df_align):
    max_cand = np.max(match_cand_score)
    if max_cand > 60:
        # fuzzy matched
        max_cand_id = match_cand[match_cand_score.index(max_cand)]
        #print (max_cand_id)
        word = df_align['model'][max_cand_id]
        return word, max_cand, max_cand_id
    else:
        return '-', 0, '-'
    
def get_error_type(t1, t2):
    if (t1 != '-') and (t2 != '-') and (t1 != t2):
        return 's'
    if (t1 != '-') and (t2 == '-'):
        return 'd'
    if (t1 == '-') and (t2 != '-'):
        return 'i'
    if (t1 != '-') and (t2 != '-') and (t1 == t2):
        return 'c'
    if t1 == '':
        return ""

/Users/irisz/opt/miniconda3/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [59]:
def get_df_align_and_wer_info(df, file_name):
    save_dir = '/users/irisz/downloads/elpis_result/' + file_name + '_wer_word_level.csv'
    manual_all = []
    model_all = []
    model_fuzzy_all = []
    error_type_all = []
    wer_all = []
    for id in range(len(df)):
        #text1 = df['text'][id]
        #text2 = df[model_name][id] # already have list
        #l1 = get_clean_word_list(text1)
        #l2 = get_clean_word_list(text2)
        l1 = df['text_clean'][id]
        l2 = df['transcription'][id]
        if l1 == []:
            l1 = [""]
        if l2 == []:
            l2 = [""]
        #print (l1, l2)
        df_align = pd.DataFrame(columns = ['manual', 'model'])
        #print (l1, l2)
        df_align['manual'] = get_aligned_list(l1, l2)[0]
        df_align['model'] = get_aligned_list(l1, l2)[1]
        df_align['score'] = get_aligned_list(l1, l2)[2]
        df_align['match'] = df_align.apply(lambda x:get_matched(x.manual, x.model), axis = 1)
        df_align['idx'] = df_align.index
        df_align['match_cand'] = df_align.apply(lambda x: get_unmatched_cand(df_align['match'].tolist(), x.idx), axis = 1)
        df_align['match_cand_score'] = df_align.apply(lambda x: get_can_score(x.idx, x.match_cand, df_align), axis = 1)
        df_align['model_fuzzy'] = df_align.apply(lambda x: get_fuzzy_cand(x.match_cand, x.match_cand_score, df_align)[0], axis = 1)
        df_align['model_fuzzy_id'] = df_align.apply(lambda x: get_fuzzy_cand(x.match_cand, x.match_cand_score, df_align)[2], axis = 1)
        df_align['model_fuzzy_score'] = df_align.apply(lambda x: get_fuzzy_cand(x.match_cand, x.match_cand_score, df_align)[1], axis = 1)
        df_align['error_type'] = df_align.apply(lambda x: get_error_type(x.manual, x.model), axis = 1)
        sub_id = df_align[(df_align['error_type']=='d') & (df_align['model_fuzzy_id'] != '-')]['model_fuzzy_id'].tolist()
        del_sub_id = df_align[(df_align['error_type']=='d') & (df_align['model_fuzzy_id'] != '-')].index.tolist()
        error_type = df_align['error_type'].tolist()
        for id in del_sub_id:
            error_type[id] = 's'
        df_align['error_type_correct'] = error_type
        df_align_clean = df_align[~df_align.index.isin(sub_id)]

        manual_curr = df_align_clean['manual'].tolist()
        model_fuzzy_curr  = df_align_clean['model_fuzzy'].tolist()
        error_type  = df_align_clean['error_type_correct'].tolist()
        model_curr = df_align_clean['model'].tolist()
        wer_curr = get_wer_from_df(df_align_clean)
        
        manual_all = manual_all + manual_curr
        model_fuzzy_all = model_fuzzy_all + model_fuzzy_curr
        error_type_all = error_type_all + error_type
        model_all = model_all + model_curr
        wer_all.append(wer_curr)
    df_all = pd.DataFrame(columns = ['manual', 'model', 'model_fuzzy', 'error_type'])
    df_all['manual'] = manual_all
    df_all['model'] = model_all
    df_all['model_fuzzy'] = model_fuzzy_all
    df_all['error_type'] = error_type_all
    
    df_all.to_csv(save_dir)
    return wer_all

def get_wer_from_df(df):
    s = len(df[df['error_type_correct'] == 's'])
    d = len(df[df['error_type_correct'] == 'd'])
    i = len(df[df['error_type_correct'] == 'i'])
    c = len(df[df['error_type_correct'] == 'c'])
    n = s + d + c
    if n == 0:
        wer = np.inf
    else:
        wer = (s + d + i)/n * 100
    return wer,s,d,i,c,n

In [66]:
# recluster sentence to 10 seconds window for wer calculation

def locate_end_small(xmin_list, xmax_list, i, chunk_size):
    
    start_curr = xmin_list[i]
    end_curr = xmax_list[i]

    if xmax_list[-1] - start_curr <= chunk_size: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1

    if end_curr - start_curr >= chunk_size:
        return end_curr, i + 1
    
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', start_curr, end_curr, i)
        if end_curr - start_curr >= chunk_size:
            
            if xmin_list[i] - start_curr >= chunk_size: # current start - last end > 10 blank, sep the blank and extend the last one
                return start_curr + chunk_size - 2,i
            else:
                return end_curr, i + 1
      
        else:
            i = i + 1
            continue

In [67]:
# chunk size 15 - 20 seconds


def get_chunk_text(start_id, end_id, file_tier, df_sent):
    df_curr = df_sent[df_sent['tier'] == file_tier] 
    text_list = df_curr['text'].tolist() # change to clean
    text_chunks = text_list[start_id : end_id]
    text = ' '.join(text_chunks)
    return text


def get_reclustered_df_bench(df_bench_1):
    chunk_size = 10 # can change chunk size

    start = []
    end = []
    start_id = []
    end_id = []
    file_info = []
    chunk_name = []
    file_tier_list = [1, 2]
    
    for file_name in file_tier_list:
        
        df_curr = df_bench_1[df_bench_1['tier'] == file_name]
        xmin_list = df_curr['xmin'].tolist()
        xmax_list = df_curr['xmax'].tolist()
    
        i = 0
        j = 0 # counting the file chunk number
        while i < len(xmin_list):
            start_curr = xmin_list[i]
            i_curr = i
            # locate the one that > 15 and  < 25 in endlist
            end_curr = locate_end_small(xmin_list, xmax_list, i, chunk_size)[0] # change locate end function
            i = locate_end_small(xmin_list, xmax_list, i, chunk_size)[1]
            #print (start_curr, end_curr, i_curr, i)
        
            j = j + 1 # chunk number update
        
            start.append(start_curr)
            end.append(end_curr)
            start_id.append(i_curr)
            end_id.append(i)
            file_info.append(file_name)
            chunk_name.append(str(file_name) + '_' + str(j))
    
    #print (file_info)    
    df_chunk = pd.DataFrame()
    df_chunk['chunk_name'] = chunk_name
    df_chunk['file_tier'] = file_info
    df_chunk['xmin'] = start
    df_chunk['xmax'] = end
    df_chunk['start_id'] = start_id
    df_chunk['end_id'] = end_id
    
    #print (df_chunk.iloc[0])
    #

    return df_chunk

In [13]:
# load files 

import glob
dir_list = glob.glob("/users/irisz/downloads/test_transcriptions_resampled/*.eaf")
dir_list


['/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_01.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_01.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_02.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_02.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_03.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_07.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_06.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_06.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_04.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_04.eaf',
 '/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_10.eaf',
 '/users/irisz/downloads/test_tr

In [68]:
for file_id in range(len(dir_list)):
    eaf_file = dir_list[file_id]

    print (eaf_file)
    file_name = eaf_file.split('/')[-1].replace('.eaf', '.TextGrid')
    print (file_name)
    
    df_all = get_df_eaf(eaf_file)
    df_bench_1 = get_bench_df(eaf_file)
    
    # change df_bench_1 word clustering to 10 seconds to match whisper results
    
    df_bench_chunk = get_reclustered_df_bench(df_bench_1)
    df_bench_chunk['text'] = df_bench_chunk.apply(lambda x: get_chunk_text(x.start_id, x.end_id, x.file_tier, df_bench_1), axis = 1)
    df_bench_chunk['text_clean'] = df_bench_chunk['text'].apply(lambda x: get_clean_word_list(x))

    # sort df_bench_chunk by xmax
    df_bench_chunk = df_bench_chunk.sort_values(by=['xmin'], ascending=True)
    df_bench_chunk.reset_index(inplace = True)
    
    merge_loc = get_merge_loc(df_bench_chunk, df_all)
    
    df_all['index'] = df_all.index
    merge_loc_all = []
    for n in range(len(df_all)):
        try:
            loc = merge_loc[n]
        except:
            loc = merge_loc[n-1]
        merge_loc_all.append(loc)
    df_all['merge_loc'] = merge_loc_all
    #df_all['merge_loc'] = df_all['index'].apply(lambda x: merge_loc[x])
    
    df_bench_chunk['index'] = df_bench_chunk.index
    df_bench_chunk['transcription'] = df_bench_chunk['index'].apply(lambda x: df_all[df_all['merge_loc']==x]['transcription'].tolist())
    df_bench_chunk['wer'] = get_df_align_and_wer_info(df_bench_chunk, file_name) # df wer word level saved 
    df_bench_chunk['wer_value'] = df_bench_chunk['wer'].apply(lambda x: x[0])

    df_bench_chunk.to_csv('/users/irisz/downloads/elpis_result/' + file_name + '_wer_value.csv')


/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_01.eaf
ckm015-2023-06-08-Klana_01.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_01.eaf
ckm016-2023-06-10-Klana_01.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_02.eaf
ckm015-2023-06-08-Klana_02.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_02.eaf
ckm016-2023-06-10-Klana_02.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_03.eaf
ckm015-2023-06-08-Klana_03.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_07.eaf
ckm016-2023-06-10-Klana_07.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm016-2023-06-10-Klana_06.eaf
ckm016-2023-06-10-Klana_06.TextGrid
/users/irisz/downloads/test_transcriptions_resampled/ckm015-2023-06-08-Klana_06.eaf
ckm015-2023-06-08-Klana_06.TextGrid
/users/irisz/downloads/test_transcriptio

In [69]:
df_bench_chunk

,index,chunk_name,file_tier,xmin,xmax,start_id,end_id,text,text_clean,transcription,wer,wer_value
0,0,1_1,1,2.224868,17.274467,0,5,Dobro. ((breath)) @ Prvo smo @ smo spominjali ...,"[dobro, prvo, smo, smo, spominjali, tako, kako...","[z, dobro, m, prvo, znan, hm, ne, znan, smo, m...","(109.09090909090908, 6, 15, 15, 12, 33)",109.090909
1,1,1_2,1,17.274467,25.274467,5,7,((breath)) @ za veće fešte?,"[za, veće, fešte]","[ja, čija, oni, je, o, o, božić, je, ja, oni, ...","(766.6666666666667, 0, 3, 20, 0, 3)",766.666667
2,2,2_1,2,18.374964,29.941305,0,1,"@ znači, na- na vilijo Božića i na vilijo Vazm...","[znači, na, na, vilijo, božića, i, na, vilijo,...","[maha, ocen, vrti, je, vani, je, to, sat, tić,...","(135.29411764705884, 0, 15, 8, 2, 17)",135.294118
3,3,2_2,2,30.048064,46.231411,1,4,Za Božić uz pasutice i bakalo je bila anke kak...,"[za, božić, uz, pasutice, i, bakalo, je, bila,...","[za, ma, božić, to, spat, kapice, jedna, to, g...","(150.0, 5, 10, 21, 9, 24)",150.000000
4,4,2_3,2,47.001732,60.114111,4,6,((sighs)) Na dan Božića smo nekat frmevali kak...,"[na, dan, božića, smo, nekat, frmevali, kako, ...","[s, šah, tu, noć, čas, bile, dan, božića, smo,...","(147.36842105263156, 6, 7, 15, 6, 19)",147.368421
5,5,1_3,1,48.496534,56.496534,7,8,"Znači, ča ste frmevali na dan Božića?","[znači, ča, ste, frmevali, na, dan, božića]",[],"(114.28571428571428, 0, 7, 1, 0, 7)",114.285714
6,6,2_4,2,60.558974,73.591283,6,11,A sada va zadnje vrieme ((breath)) je vajka ne...,"[a, sada, va, zadnje, vrieme, je, vajka, neko,...","[da, se, dala, zadnje, vrime, je, bajke, neko,...","(100.0, 7, 10, 13, 13, 30)",100.000000
7,7,2_5,2,73.591283,84.080565,11,15,"((breath)) ma, ni baš neko pravilo da muora ((...","[ma, ni, baš, neko, pravilo, da, muora, bit, v...","[na, ni, baš, neku, pranono, damu, na, kao, bi...","(105.26315789473684, 6, 4, 10, 9, 19)",105.263158
8,8,2_6,2,84.080565,96.562373,15,18,"znači, mat i otac i moji dva brata, ((breath))...","[znači, mat, i, otac, i, moji, dva, brata, a, ...","[mat, je, placi, moji, dva, brata, a, ako, za,...","(130.0, 0, 11, 15, 9, 20)",130.000000
9,9,2_7,2,96.562373,104.562373,18,21,((breath)) muški jako vole janca. Ja nisan jak...,"[muški, jako, vole, janca, ja, nisan, jako, za...","[još, ki, jako, volin, ja, nan, znan, ja, nisa...","(141.1764705882353, 7, 5, 12, 5, 17)",141.176471


In [70]:
df_all

,start,end,transcription,confidence,index,merge_loc
0,1.56,1.630,z,0.39,0,0
1,2.25,2.860,dobro,1.00,1,0
2,5.18,5.340,m,1.00,2,0
3,5.34,5.640,prvo,1.00,3,0
4,5.64,5.850,znan,0.49,4,0
...,...,...,...,...,...,...
903,385.48,385.830,eh,1.00,903,36
904,385.83,386.010,n,1.00,904,37
905,386.05,386.310,je,0.86,905,37
906,386.32,386.599,pun,0.86,906,37
